## 1- Installing prerequisites and cloning the project

In [1]:
!pip install -q lmdb fire gdown
!git clone https://github.com/clovaai/deep-text-recognition-benchmark.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 5.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.8/297.8 kB 14.6 MB/s eta 0:00:00
Cloning into 'deep-text-recognition-benchmark'...
remote: Enumerating objects: 499, done.
remote: Counting objects: 100% (225/225), done.
remote: Compressing objects: 100% (25/25), done.
remote: Total 499 (delta 208), reused 200 (delta 200), pack-reused 274 (from 1)
Receiving objects: 100% (499/499), 3.05 MiB | 30.08 MiB/s, done.
Resolving deltas: 100% (308/308), done.


## 2- Downloading pretrained weights and dataset

In [2]:
# دانلود مدل آموزش‌دیده
!gdown 1ubkg7E2vGEOqS4K_quwf9Vl-i8IVpklM  # مدل .pth
!gdown 1AL5Zsg2hDqcwF8ZmR0MJTbjgXIoE5W-I  # دوتا فایل zip

# باز کردن zip ها
!unzip -q /content/plate_img-train.zip -d /content/Dataset/
!unzip -q /content/plate_img-validation.zip -d /content/Dataset/


Downloading...
From (original): https://drive.google.com/uc?id=1ubkg7E2vGEOqS4K_quwf9Vl-i8IVpklM
From (redirected): https://drive.google.com/uc?id=1ubkg7E2vGEOqS4K_quwf9Vl-i8IVpklM&confirm=t&uuid=fb235396-02a8-4077-adb0-c9996dca4ed6
To: /content/plate_img-train.zip
100% 196M/196M [00:01<00:00, 173MB/s]
Downloading...
From (original): https://drive.google.com/uc?id=1AL5Zsg2hDqcwF8ZmR0MJTbjgXIoE5W-I
From (redirected): https://drive.google.com/uc?id=1AL5Zsg2hDqcwF8ZmR0MJTbjgXIoE5W-I&confirm=t&uuid=717df15a-d94c-4c12-9480-441cd8141377
To: /content/plate_img-validation.zip
100% 27.2M/27.2M [00:00<00:00, 47.5MB/s]


## 3- Preparing GT (Ground Truth) files for training

In [3]:
import os
import xml.etree.ElementTree as ET

train_dir = '/content/Dataset/train'
output_file = '/content/deep-text-recognition-benchmark/data/gt_train.txt'
os.makedirs(os.path.dirname(output_file), exist_ok=True)
lines = []

for filename in os.listdir(train_dir):
    if filename.endswith('.xml'):
        xml_path = os.path.join(train_dir, filename)
        tree = ET.parse(xml_path)
        root = tree.getroot()

        image_name = root.find('filename').text
        image_path = os.path.join(train_dir, image_name)

        chars = []
        for obj in root.findall('object'):
            name = obj.find('name').text
            xmin = int(obj.find('bndbox').find('xmin').text)
            chars.append((xmin, name))

        chars.sort(key=lambda x: x[0])
        label = ''.join([char[1] for char in chars])
        lines.append(f'{image_path}\t{label}')

with open(output_file, 'w', encoding='utf-8') as f:
    f.write('\n'.join(lines))

print(f'Done! {len(lines)} items written to {output_file}')


Done! 19381 items written to /content/deep-text-recognition-benchmark/data/gt_train.txt


سلول 4 – آماده‌سازی فایل‌های GT برای validation

In [4]:
valid_dir = '/content/Dataset/validation'
output_file = '/content/deep-text-recognition-benchmark/data/gt_validation.txt'
lines = []

for filename in os.listdir(valid_dir):
    if filename.endswith('.xml'):
        xml_path = os.path.join(valid_dir, filename)
        tree = ET.parse(xml_path)
        root = tree.getroot()

        image_name = root.find('filename').text
        image_path = os.path.join(valid_dir, image_name)

        chars = []
        for obj in root.findall('object'):
            name = obj.find('name').text
            xmin = int(obj.find('bndbox').find('xmin').text)
            chars.append((xmin, name))

        chars.sort(key=lambda x: x[0])
        label = ''.join([char[1] for char in chars])
        lines.append(f'{image_path}\t{label}')

with open(output_file, 'w', encoding='utf-8') as f:
    f.write('\n'.join(lines))

print(f'Done! {len(lines)} items written to {output_file}')


Done! 2805 items written to /content/deep-text-recognition-benchmark/data/gt_validation.txt


سلول 5 – جابه‌جایی تصاویر به مسیر data/train و data/validation

In [5]:
import shutil

# Move train images
src = '/content/Dataset/train'
dst = '/content/deep-text-recognition-benchmark/data/train'
os.makedirs(dst, exist_ok=True)

for file in os.listdir(src):
    if file.endswith('.jpg'):
        shutil.move(os.path.join(src, file), os.path.join(dst, file))

# Move validation images
src = '/content/Dataset/validation'
dst = '/content/deep-text-recognition-benchmark/data/validation'
os.makedirs(dst, exist_ok=True)

for file in os.listdir(src):
    if file.endswith('.jpg'):
        shutil.move(os.path.join(src, file), os.path.join(dst, file))

print("Train and validation images moved.")


Train and validation images moved.


🟢 سلول 6 – ساخت LMDB دیتاست

In [9]:
%cd /content/deep-text-recognition-benchmark

!python3 create_lmdb_dataset.py --inputPath data/train --gtFile data/gt_train.txt --outputPath dataset/train
!python3 create_lmdb_dataset.py --inputPath data/validation --gtFile data/gt_validation.txt --outputPath dataset/validation

Streaming output truncated to the last 5000 lines.
/content/Dataset/train/for_xml_130628.jpg does not exist
/content/Dataset/train/for_xml_2274.jpg does not exist
/content/Dataset/train/for_xml_134629.jpg does not exist
/content/Dataset/train/for_xml_117267.jpg does not exist
/content/Dataset/train/for_xml_2160.jpg does not exist
/content/Dataset/train/for_xml_121891.jpg does not exist
/content/Dataset/train/for_xml_128.jpg does not exist
/content/Dataset/train/for_xml_135544.jpg does not exist
/content/Dataset/train/for_xml_135490.jpg does not exist
/content/Dataset/train/for_xml_4632.jpg does not exist
/content/Dataset/train/for_xml_642.jpg does not exist
/content/Dataset/train/for_xml_135128.jpg does not exist
/content/Dataset/train/for_xml_131765.jpg does not exist
/content/Dataset/train/for_xml_117077.jpg does not exist
/content/Dataset/train/for_xml_4020.jpg does not exist
/content/Dataset/train/for_xml_198.jpg does not exist
/content/Dataset/train/for_xml_125689.jpg does not exi

In [ ]:
!python3 train.py \
--train_data dataset/train --valid_data dataset/validation \
--select_data / --batch_ratio 1 --batch_max_length 8 --valInterval 100 \
--Transformation TPS --FeatureExtraction ResNet --SequenceModeling BiLSTM --Prediction Attn


/content/
│
├── Dataset/
│   ├── train/
│   │   ├── *.jpg
│   │   └── *.xml
│   └── validation/
│       ├── *.jpg
│       └── *.xml
│
└── deep-text-recognition-benchmark/
    ├── data/
    │   ├── gt_train.txt
    │   ├── gt_validation.txt
    │   ├── train/         ← عکس‌های train
    │   └── validation/    ← عکس‌های validation
    └── dataset/
        ├── train/         ← LMDB ساخته شده
        └── validation/    ← LMDB ساخته شده
